In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd #import pandas

In [2]:
mongo = MongoClient(port=27017)

In [3]:
db = mongo['jobs_in_data']

In [4]:
# review the collections in our database
print(db.list_collection_names())

['jobs']


In [5]:
establishments = db['jobs']

In [6]:
query = {'salary' : 100000}

# Use count_documents to display the number of documents in the result
result = establishments.count_documents(query)
# Display the first document in the results using pprint
pprint(result)

203
